In [1]:
import pandas as pd

path_to_train = './data/train.csv'
path_to_test = './data/test.csv'

train = pd.read_csv(path_to_train, encoding='utf-8', usecols=['user_id', 'ts_listen', 'listen_type', 'is_listened'])
test = pd.read_csv(path_to_test, encoding='utf-8', usecols=['user_id', 'ts_listen', 'listen_type'])
test['is_listened'] = 0 # просто щоб зібрати, ми будемо використовувати попередні значення

df = pd.concat([train, test], ignore_index=True)
df.tail()

,ts_listen,listen_type,user_id,is_listened
7578747,1479664431,1,12641,0
7578748,1480358099,1,10055,0
7578749,1480616853,1,1029,0
7578750,1479644510,1,4630,0
7578751,1480491885,1,6467,0


In [2]:
df = df.sort_values(by=['user_id', 'ts_listen'], axis=0)
df.tail()

,ts_listen,listen_type,user_id,is_listened
7575803,1478016326,1,19915,0
569368,1477512816,0,19916,1
7563826,1478017434,1,19916,0
4791398,1477926978,0,19917,1
7565747,1478009685,1,19917,0


In [3]:
df.head()

,ts_listen,listen_type,user_id,is_listened
682493,1477939775,0,0,1
4454939,1477982531,0,0,0
5354523,1477982561,0,0,0
5899458,1477982582,0,0,1
4716574,1477982792,0,0,1


In [4]:
df['cumsum_listen_type'] = (df['listen_type'].groupby((
                (df[['listen_type','user_id']].shift() != df[['listen_type','user_id']]).any(axis=1)
            ).cumsum()).cumcount() + 1).shift()

df['cumsum_listen_type_1'] = (df['listen_type'] * (df['listen_type'].groupby((
                (df[['listen_type','user_id']].shift() != df[['listen_type','user_id']]).any(axis=1)
            ).cumsum()).cumcount() + 1)).shift()

fi = df.groupby('user_id', as_index=False).apply(lambda x: x.index[0])
df.ix[fi, 'cumsum_listen_type_1'] = 0
df.ix[fi, 'cumsum_listen_type_0'] = 0
df.ix[fi, 'cumsum_listen_type'] = 0

df[['user_id', 'listen_type', 'cumsum_listen_type', 'cumsum_listen_type_1']].tail(10)

,user_id,listen_type,cumsum_listen_type,cumsum_listen_type_1
2549309,19913,0,0.0,0.0
7575054,19913,1,1.0,0.0
2857394,19914,0,0.0,0.0
7573591,19914,1,1.0,0.0
4242361,19915,1,0.0,0.0
7575803,19915,1,1.0,1.0
569368,19916,0,0.0,0.0
7563826,19916,1,1.0,0.0
4791398,19917,0,0.0,0.0
7565747,19917,1,1.0,0.0


In [5]:
df[['user_id', 'listen_type', 'cumsum_listen_type', 'cumsum_listen_type_1']].head(10)

,user_id,listen_type,cumsum_listen_type,cumsum_listen_type_1
682493,0,0,0.0,0.0
4454939,0,0,1.0,0.0
5354523,0,0,2.0,0.0
5899458,0,0,3.0,0.0
4716574,0,0,4.0,0.0
2859238,0,0,5.0,0.0
3796053,0,0,6.0,0.0
1209271,0,0,7.0,0.0
6888438,0,0,8.0,0.0
6714892,0,0,9.0,0.0


In [6]:
df['cumsum_is_listened'] = (df['is_listened'].groupby((
                (df[['is_listened','user_id']].shift() != df[['is_listened','user_id']]).any(axis=1)
            ).cumsum()).cumcount() + 1).shift()

df['cumsum_is_listened_1'] = (df['is_listened'] * (df['is_listened'].groupby((
                (df[['is_listened','user_id']].shift() != df[['is_listened','user_id']]).any(axis=1)
            ).cumsum()).cumcount() + 1)).shift()


fi = df.groupby('user_id', as_index=False).apply(lambda x: x.index[0])
df.ix[fi, 'cumsum_is_listened_1'] = 0
df.ix[fi, 'cumsum_is_listened'] = 0

df[['user_id', 'is_listened', 'cumsum_is_listened', 'cumsum_is_listened_1']].tail(10)

,user_id,is_listened,cumsum_is_listened,cumsum_is_listened_1
2549309,19913,1,0.0,0.0
7575054,19913,0,1.0,1.0
2857394,19914,1,0.0,0.0
7573591,19914,0,1.0,1.0
4242361,19915,1,0.0,0.0
7575803,19915,0,1.0,1.0
569368,19916,1,0.0,0.0
7563826,19916,0,1.0,1.0
4791398,19917,1,0.0,0.0
7565747,19917,0,1.0,1.0


In [7]:
df[['user_id', 'is_listened', 'cumsum_is_listened', 'cumsum_is_listened_1']].head(10)

,user_id,is_listened,cumsum_is_listened,cumsum_is_listened_1
682493,0,1,0.0,0.0
4454939,0,0,1.0,1.0
5354523,0,0,1.0,0.0
5899458,0,1,2.0,0.0
4716574,0,1,1.0,1.0
2859238,0,1,2.0,2.0
3796053,0,1,3.0,3.0
1209271,0,1,4.0,4.0
6888438,0,1,5.0,5.0
6714892,0,1,6.0,6.0


In [8]:
# user_max = df.groupby('user_id', as_index=False).agg({
#         'cumsum_listen_type':'max',
#         'cumsum_listen_type_1':'max',
#         'cumsum_listen_type_0':'max',
#         'cumsum_is_listened':'max',
#         'cumsum_is_listened_1':'max',
#         'cumsum_is_listened_0':'max',
#     })

# df = df.join(user_max, on='user_id', rsuffix='_max')
# df.head()

# normalization
df.ix[df['cumsum_is_listened'] > 5, 'cumsum_is_listened'] = 5
df.ix[df['cumsum_is_listened_1'] > 5, 'cumsum_is_listened_1'] = 5
df.ix[df['cumsum_listen_type'] > 5, 'cumsum_listen_type'] = 5
df.ix[df['cumsum_listen_type_1'] > 5, 'cumsum_listen_type_1'] = 5
df.head()

,ts_listen,listen_type,user_id,is_listened,cumsum_listen_type,cumsum_listen_type_1,cumsum_listen_type_0,cumsum_is_listened,cumsum_is_listened_1
682493,1477939775,0,0,1,0.0,0.0,0.0,0.0,0.0
4454939,1477982531,0,0,0,1.0,0.0,NaN,1.0,1.0
5354523,1477982561,0,0,0,2.0,0.0,NaN,1.0,0.0
5899458,1477982582,0,0,1,3.0,0.0,NaN,2.0,0.0
4716574,1477982792,0,0,1,4.0,0.0,NaN,1.0,1.0


In [9]:
# df['cumsum_is_listened'] = df['cumsum_is_listened']/df['cumsum_is_listened_max']
# df['cumsum_is_listened_1'] = df['cumsum_is_listened_1']/df['cumsum_is_listened_1_max']
# df['cumsum_is_listened_0'] = df['cumsum_is_listened_0']/df['cumsum_is_listened_0_max']

# df['cumsum_listen_type'] = df['cumsum_listen_type']/df['cumsum_listen_type_max']
# df['cumsum_listen_type_1'] = df['cumsum_listen_type_1']/df['cumsum_listen_type_1_max']
# df['cumsum_listen_type_0'] = df['cumsum_listen_type_0']/df['cumsum_listen_type_0_max']

# df.head(10)

In [10]:
# del df['user_id_max']
# del df['count']

In [10]:
df = df.sort_index()
df.tail()

,ts_listen,listen_type,user_id,is_listened,cumsum_listen_type,cumsum_listen_type_1,cumsum_listen_type_0,cumsum_is_listened,cumsum_is_listened_1
7578747,1479664431,1,12641,0,5.0,0.0,NaN,1.0,1.0
7578748,1480358099,1,10055,0,3.0,3.0,NaN,5.0,5.0
7578749,1480616853,1,1029,0,5.0,5.0,NaN,5.0,5.0
7578750,1479644510,1,4630,0,5.0,0.0,NaN,5.0,5.0
7578751,1480491885,1,6467,0,5.0,5.0,NaN,3.0,3.0


In [11]:
path_to_test_cumsum = './data/features/cumsum.test.csv'
path_to_train_cumsum = './data/features/cumsum.train.csv'
new_cols = [
    'cumsum_listen_type', 
    'cumsum_listen_type_1', 
#     'cumsum_listen_type_0', 
    'cumsum_is_listened', 
    'cumsum_is_listened_1', 
#     'cumsum_is_listened_0',
#     'cumsum_listen_type_max', 
#     'cumsum_listen_type_1_max', 
#     'cumsum_listen_type_0_max', 
#     'cumsum_is_listened_max', 
#     'cumsum_is_listened_1_max', 
#     'cumsum_is_listened_0_max'
]

In [12]:
test_cumsum = df.iloc[range(len(train), len(train)+len(test))][new_cols]
test_cumsum.to_csv(path_to_test_cumsum, encoding='utf-8', index=False)

train_cumsum = df.iloc[range(len(train))][new_cols]
train_cumsum.to_csv(path_to_train_cumsum, encoding='utf-8', index=False)

-------------

In [2]:
import validation as v
import pandas as pd

path_to_train = './data/train.csv'

train = pd.read_csv(path_to_train, encoding='utf-8')
train_cumsum = pd.read_csv(path_to_train_cumsum, encoding='utf-8')

train = pd.concat([train, train_cumsum], axis=1)
train.head()

,genre_id,ts_listen,media_id,album_id,context_type,release_date,platform_name,platform_family,media_duration,listen_type,...,cumsum_listen_type_0,cumsum_is_listened,cumsum_is_listened_1,cumsum_is_listened_0,cumsum_listen_type_max,cumsum_listen_type_1_max,cumsum_listen_type_0_max,cumsum_is_listened_max,cumsum_is_listened_1_max,cumsum_is_listened_0_max
0,25471,1480597215,222606,41774,12,20040704,1,0,223,0,...,0.173077,0.125000,0.000000,0.333333,52.0,37.0,52.0,24.0,24.0,9.0
1,25571,1480544735,250467,43941,0,20060301,2,1,171,0,...,0.114286,0.100000,0.100000,0.000000,35.0,1.0,35.0,10.0,10.0,4.0
2,16,1479563953,305197,48078,1,20140714,2,1,149,1,...,0.027778,0.878049,0.878049,0.000000,72.0,24.0,72.0,123.0,123.0,1.0
3,7,1480152098,900502,71521,0,20001030,0,0,240,0,...,0.539326,0.006494,0.000000,0.023810,267.0,125.0,267.0,154.0,154.0,42.0
4,7,1478368974,542335,71718,0,20080215,0,0,150,0,...,0.277372,0.108571,0.108571,0.000000,274.0,79.0,274.0,175.0,175.0,8.0


In [3]:
#import preparation as p
#train = p.minimal_entries_by_user(train, 2)
#train = p.remove_percent_entries_for_users_who_have_to_much(train, 3000, 0.4)

/home/tsdaemon/anaconda3/lib/python3.6/site-packages/pandas/indexes/base.py:1428: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  return promote(getitem(key))


In [4]:
len(train)

7411160

In [5]:
path_to_folds = './data/lightgbm/cumsum'

In [6]:
v.generate_validation_folds(train, path_to_folds)

Fold 1


In [10]:
import model_lgbm as mlgbm

new_cols = [
    'cumsum_listen_type', 
    'cumsum_listen_type_1', 
    'cumsum_listen_type_0',
    'cumsum_listen_type_max', 
    'cumsum_listen_type_1_max', 
    'cumsum_listen_type_0_max', 
    
    #'cumsum_is_listened', 
    #'cumsum_is_listened_1', 
    #'cumsum_is_listened_0',
    #'cumsum_is_listened_max', 
    #'cumsum_is_listened_1_max', 
    #'cumsum_is_listened_0_max'
]

cols = [
    'genre_id', 
    'media_id', 
    'album_id', 
    'context_type',
    'release_date', 
    'platform_name', 
    'platform_family', 
    'media_duration',
    'listen_type',
    'user_gender', 
    'user_id',
    'artist_id', 
    'user_age'
] + new_cols

params = {
    'application':'binary',
    'num_leaves': 24,
    'max_depth': 15,
    'feature_fraction': 0.7,
    'bagging_fraction': 0.7,
    'max_bin': 80,
    'metric': 'auc',
    'verbose': 1
}

In [11]:
evs = mlgbm.crossvalidate_model(path_to_folds,cols,params,validate_on_both=True, iterations=40)

[1]	training's auc: 0.650755	valid_1's auc: 0.555916
[2]	training's auc: 0.653741	valid_1's auc: 0.532767
[3]	training's auc: 0.669051	valid_1's auc: 0.513462
[4]	training's auc: 0.670716	valid_1's auc: 0.503062
[5]	training's auc: 0.672325	valid_1's auc: 0.505214
[6]	training's auc: 0.671672	valid_1's auc: 0.506289
[7]	training's auc: 0.673378	valid_1's auc: 0.506816
[8]	training's auc: 0.673807	valid_1's auc: 0.514435
[9]	training's auc: 0.673904	valid_1's auc: 0.521693
[10]	training's auc: 0.675093	valid_1's auc: 0.521125
[11]	training's auc: 0.67577	valid_1's auc: 0.521746
[12]	training's auc: 0.676055	valid_1's auc: 0.522864
[13]	training's auc: 0.676691	valid_1's auc: 0.527456
[14]	training's auc: 0.677159	valid_1's auc: 0.526979
[15]	training's auc: 0.677537	valid_1's auc: 0.526947
[16]	training's auc: 0.678179	valid_1's auc: 0.527581
[17]	training's auc: 0.678771	valid_1's auc: 0.527492
[18]	training's auc: 0.679225	valid_1's auc: 0.529794
[19]	training's auc: 0.680146	valid_1'

In [12]:
mlgbm.feature_score(evs[1],cols)

[('cumsum_listen_type_1_max', 187),
 ('context_type', 153),
 ('cumsum_listen_type_0_max', 134),
 ('cumsum_listen_type_max', 124),
 ('user_age', 110),
 ('platform_name', 59),
 ('user_gender', 36),
 ('platform_family', 24),
 ('media_duration', 21),
 ('listen_type', 17),
 ('release_date', 17),
 ('genre_id', 16),
 ('cumsum_listen_type_0', 9),
 ('cumsum_listen_type_1', 9),
 ('cumsum_listen_type', 4)]

In [ ]:
path_to_sample = './data/sample_submission_kaggle.csv'
path_to_submission = './data/submission.csv'
path_to_test = './data/test.csv'

test = pd.read_csv(path_to_test, encoding='utf-8')
test = pd.concat([test, pd.read_csv(path_to_feature_test, encoding='utf-8')], axis=1)

test.head()

In [ ]:
fs = mlgbm.create_submission(train, test, path_to_sample, params, cols, path_to_submission, iterations=400)

In [ ]:
fscore = list(zip(cols, fs))
fscore.sort(key=lambda x: x[1])
fscore.reverse()
fscore